In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
#part 1 (lets load our framework (for this note book its tf)(i wont repeat this))
import tensorflow as tf
#tensorflow
from tensorflow.keras import layers,Sequential,Model,utils
import tensorflow_hub as hub
#we will use it later

In [8]:
#part 1 (adding some callbacks)
earlystopping=tf.keras.callbacks.EarlyStopping(
    monitor='Accuracy',
    min_delta=0.0001,
    patience=2,
    restore_best_weights=True,
    start_from_epoch=5
)

tensorboard= tf.keras.callbacks.TensorBoard()
chkpt = tf.keras.callbacks.ModelCheckpoint(
    'mango.ckpt',
    monitor = 'Accuracy',
    save_best_only = True,
    
    mode = 'auto',
    save_freq='epoch',
)


In [9]:
#part 2(loading and normailizing tha data)
# since the data is so large i will use tf.keras.utils.image_dataset_from_directory and also i will only load it as 128,128,3(for storage ressons)
# and also i will normalize the data by going through evey single one and normalizing it as you will see below
df = tf.keras.utils.image_dataset_from_directory('/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset',
                                            image_size=(128,128),
                                            batch_size=16)
# this is the normalization layer
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

df = df.map(process)
#Ds means data set btw

Found 5945 files belonging to 40 classes.


In [10]:
#part 2(building the model)
# for this model i will use the eff_net b7
model_plants = Sequential([
    layers.Input(shape=(128,128,3)),
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b7/classification/1"),
    #the layer which loads eff_net b7
    layers.Dense(128,activation="relu"),
    layers.Dense(40,activation='softmax')
])

In [11]:
model_plants.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              66658680  
                                                                 
 dense (Dense)               (None, 128)               128128    
                                                                 
 dense_1 (Dense)             (None, 40)                5160      
                                                                 
Total params: 66,791,968
Trainable params: 133,288
Non-trainable params: 66,658,680
_________________________________________________________________


In [12]:
#part 2 (compiling the model)
#we will use Adam as an optimizer sparse catagorical as our loss and metrics is accuracy
model_plants.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.02),
    metrics=['Accuracy']
)

In [13]:
 with tf.device('/GPU:0'):
    model_plants.fit(df,
              epochs=50,
              callbacks=[earlystopping,tensorboard,chkpt])

Epoch 1/50
372/372 [==============================] - 131s 243ms/step - loss: 2.8380 - Accuracy: 0.2308
Epoch 2/50
372/372 [==============================] - 27s 72ms/step - loss: 1.8922 - Accuracy: 0.4705
Epoch 3/50
372/372 [==============================] - 27s 72ms/step - loss: 1.4688 - Accuracy: 0.5798
Epoch 4/50
372/372 [==============================] - 28s 76ms/step - loss: 1.2246 - Accuracy: 0.6421
Epoch 5/50
372/372 [==============================] - 28s 74ms/step - loss: 1.0514 - Accuracy: 0.6885
Epoch 6/50
372/372 [==============================] - 29s 76ms/step - loss: 0.9170 - Accuracy: 0.7354
Epoch 7/50
372/372 [==============================] - 28s 73ms/step - loss: 0.8288 - Accuracy: 0.7499
Epoch 8/50
372/372 [==============================] - 28s 75ms/step - loss: 0.7374 - Accuracy: 0.7791


In [19]:
# Save the model to a file
model_plants.save('model_plants.h5')


In [16]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# Load and preprocess the image
image_path = "/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Aloevera/339.jpg"
image = load_img(image_path, target_size=(128, 128))
image = img_to_array(image)
image = image / 255.0  # Normalize pixel values to [0, 1]
image = np.expand_dims(image, axis=0)  # Add batch dimension


In [18]:
predictions = model_plants.predict(image)
predicted_class = np.argmax(predictions[0])

print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 45ms/step
Predicted class: 0
